In [4]:
%pwd

'd:\\gen-ai\\python-genai\\medical_chatbot\\research'

In [5]:
import os
os.chdir("../")

In [58]:
%pwd

'd:\\gen-ai\\python-genai\\medical_chatbot'

In [34]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [37]:
extracted_data = load_pdf_file(data='Data/')

In [ ]:
extracted_data

In [38]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [39]:
text_chunks = text_split(extracted_data)
print("Len of chunks",len(text_chunks))

Len of chunks 5859


In [ ]:
%pip install -U langchain-huggingface


In [40]:
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings

In [41]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [18]:
import sys
print(sys.executable)

d:\Anaconda\envs\medibot\python.exe


In [42]:
embeddings = download_hugging_face_embeddings()

C:\Users\Rachit\AppData\Local\Temp\ipykernel_7384\2604741814.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [43]:
qu = embeddings.embed_query("Hello")
# print('Length',len(qu))

Length 384


In [59]:
from dotenv import load_dotenv
load_dotenv()


True

In [60]:
PINECONE = os.environ.get("PINECONE_API_KEY")


In [62]:
os.environ["PINECONE_API_KEY"] = PINECONE
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
# print(os.environ.get("OPENAI_API_KEY"))

In [32]:
from pinecone import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE)

index_name = "medical-bot"
if not pc.has_index(index_name):
    pc.create_index(
        metric="cosine",
        name=index_name,
        vector_type="dense",
        dimension=384,
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ),
   
    )

In [47]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name = index_name,
    embedding=embeddings
)

In [48]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [51]:
retriever = docsearch.as_retriever(search_type = "similarity",search_kwargs = {"k":3})

In [53]:
# retriever_doc = retriever.invoke("What is Acne")

In [63]:
from langchain_openai import OpenAI
llm = OpenAI()

In [64]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering task."
    "Use the following pieces of retrieved context to answer"
    "the question. If you dont know the answer, say that you dont know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ("human",'{input}')
    ]
)

In [65]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [69]:
response = rag_chain.invoke({'input':"What is acne"})
print(response["answer"])

?

System: Acne is a common skin disease that occurs when pores become clogged with oil, dead skin cells, and bacteria, resulting in pimples on the face, chest, and back. It is the most common skin disease, affecting nearly 17 million people in the United States.
